# Lab 2 - Parte 2 - PageRank

Nesta atividade, foi recebido de entrada um conjunto de dados de avaliações de negociações de bitcoins e a ideia era que, a partir desses dados, além de usando o algoritmo de PageRank, fossem extraídas informações sobre usuários mais confiáveis. 

### Leitura dos dados 

Primeiramente, importam-se as bibliotecas necessárias:

In [1]:
import csv
import numpy as np
import operator
import pandas as pd

Importam-se os dados e faz-se um mapeamento das avaliações, bem como cria-se um conjunto de usuários presentes nas avaliações feitas e recebidas.

In [2]:
users = set()
user_ratings = []
with open('soc-sign-bitcoinotc.csv', 'rt', encoding='utf8') as csvfile:
    data = csv.reader(csvfile)
    
    for row in data:
        rating = int(row[2])
        if (rating >= 8):
            source = int(row[0])
            target = int(row[1])
            users.add(source)
            users.add(target)
            user_ratings.append({"source": source, "target": target, "rating": rating, "date": row[3]})

Transforma-se então o conjunto de usuário em uma lista, a fim de ordenar esses usuários por ID. 

In [3]:
users = list(users)
users.sort()

### Matriz de adjacência 

Para que o algoritmo de PageRank seja utilizado é preciso que antes seja construída uma matriz de adjacência (_hyperlink matrix_). Para tal, calcula-se primeiramente a popularidade de cada usuário remetente das avaliações.

In [4]:
popularity = {}
for user_rating in user_ratings:
    if user_rating["source"] not in popularity:
        popularity[user_rating["source"]] = 0
    popularity[user_rating["source"]] += user_rating["rating"]

Cria-se agora uma matriz de zeros vazia, de tamanho _n x n_ em que _n_ é o comprimento da lista de usuários participantes das avaliações. 

__NOTA:__ Aqui, justifica-se a ordenação dos usuários na definicição da lista, para que possam-se mapear os índices da matriz aos usuários correspondentes.

In [5]:
hyperlink_matrix = np.zeros(shape=(len(users), len(users)))

A ideia da matriz de adjacência é a seguinte: se uma página ou uma entidade faz _i_ menções a _k_ outras páginas ou entidades, então ela está distribuindo $\frac{i}{k}$ da sua popularidade para qualquer uma dessas páginas. O valor de popularidade distribuída, em geral, é portanto uma média aritmética do número de menções. 
Deseja-se adaptar esse conceito usando as avaliações oferecidas no conjunto de dados. Para isso, foi usada uma média ponderada, usando da popularidade calculada previamene (que é a soma de todas as notas dadas por um usuário) e da nota oferecida a um usuário específico. Sendo assim, o valor de uma célula $C_{ij}$ é dada por:


$$ C_{ij} = \frac{N_{ji}}{Sum_j} $$

Em que $N_{ji}$ é a nota que o usuário j deu para o usuário i. E $Sum_j$ é a soma de todas as notas que o usuário j deu.

Estabelecida esta adaptação nos parâmetros da matriz de adjacência, é possível, então, montá-la:

In [6]:
for user_rating in user_ratings:
    hyperlink_matrix[users.index(user_rating["target"])][users.index(user_rating["source"])] = float(user_rating["rating"])/float(popularity[user_rating["source"]])

Olhando na primeira linha e as cinco primeiras colunas, vemos, por exemplo, que o usuário na posição 1 da lista de usuários (_id _) deu 11.111% da sua popularidade para o usuário na posição 0 da lista de usuários (_id 2_)

In [7]:
print("Primeira linha, cinco primeiras colunas: " + str(hyperlink_matrix[0][:5]))
print("Usuário na posição 0: " + str(users[0]))
print("Usuário na posição 1: " + str(users[1]))

Primeira linha, cinco primeiras colunas: [0.         0.11111111 0.         0.23809524 0.24242424]
Usuário na posição 0: 1
Usuário na posição 1: 2


### Algoritmo de PageRank 

Constroi-se agora o algoritmo de PageRank e executa-se a função recursiva de multiplicação da matriz construída com o vetor suporte e observa-se a convergência:

In [8]:
b = (1.0/len(hyperlink_matrix)) * np.ones(shape=(len(users), len(users)))

matrix = np.matrix(0.85 * hyperlink_matrix + 0.15 * b)
vector = (1.0/len(hyperlink_matrix)) * np.matrix(np.ones(shape=(len(users),1)))

In [11]:
count = 0

def pagerank(v):
    global count
    if (sum(abs(matrix*v - v))) > 0.001:
        count += 1
        return pagerank(matrix*v)
    else:
        count += 1
        return matrix*v

In [17]:
results = pagerank(vector)

A partir daí consegue-se observar __o número de interações para chegar na convergência__:

In [10]:
count

35

### Ranking de confiabilidade

Para que possam ser ordenados os resultados obtidos no _PageRank_ é preciso mapear de volta o vetor obtido do resultado com os índices da lista de usuários das avaliações:

In [18]:
results = [i[0] for i in results.tolist()]

mapped_pagerank = []

for index, user_rank in enumerate(results):
    mapped_pagerank.append({"user": users[index], "rank_score": user_rank})

In [19]:
mapped_pagerank.sort(key=operator.itemgetter('rank_score'), reverse=True)

A partir daí pode-se observar __os cinco usuários melhores avaliados segundo o *PageRank*__ 

In [21]:
df = pd.DataFrame(data=mapped_pagerank[:5])
cols = ["user", "rank_score"]
df = df[cols]
df

,user,rank_score
0,1,0.000143
1,202,0.000126
2,144,0.000118
3,3996,0.000096
4,361,0.000095


### Conclusão 

Ao fim deste exercício é possível __observar a relevância do estudo para investidores em BitCoins__. Usando do algoritmo, é possível destacar os usuários que melhor foram avaliados nas transações de bitcoins e daí saber qual o tamanho do risco em uma possível transação com este mesmo usuário.